In [1]:
import tensorflow as tf
from tensorflow import keras
from google.colab import files
import pandas as pd
import io

uploaded = files.upload()
trip_advisor = pd.read_csv(io.BytesIO(uploaded['deceptive-opinion.csv']))
trip_advisor.describe()

Saving deceptive-opinion.csv to deceptive-opinion.csv


,deceptive,hotel,polarity,source,text
count,1600,1600,1600,1600,1600
unique,2,20,2,3,1596
top,truthful,sofitel,negative,MTurk,My daughter and I woke in the morning wanting ...
freq,800,80,800,800,2


In [2]:
import numpy as np
from keras.preprocessing.text import text_to_word_sequence

# The dataset is perfectly balanced, so 50% accuracy will be equivalent to a random guess. 
labels = np.where(trip_advisor['deceptive']=='truthful',0,1)

text = []
for i in range(len(trip_advisor)):
  text.append(text_to_word_sequence(trip_advisor['text'][i])) # This strips punctuation, odd characters, and makes things lower-case. 

Now, let's strip out infrequent terms.

In [5]:
# Let's make a running total of each term's freuqency in the data.
word_freq = {}
for review in text:
  for term in review:
    try:
        word_freq[term] = word_freq[term]+1
    except KeyError:
        word_freq[term] = 1

# This is where we can play with how frequent a word has to be to enter our model.
min_freq = 1
max_freq = max(i for i in word_freq.values())
for i in range(len(text)):
  text[i] = {term for term in text[i] if word_freq[term] >= min_freq & word_freq[term] <= max_freq}

unique_terms = {term for review in text for term in review}
print(f'We have {len(unique_terms)} unique tokens in our dataset.')

# Assign an integer to each unique term.
word_index = {term: number for number, term in enumerate(unique_terms)}
reverse_index = {number: term for number, term in enumerate(unique_terms)}

We have 10275 unique tokens in our dataset.


One-hot encoding the text.

In [6]:
def vectorize_sequences(sequences, dimension=len(unique_terms)): 
    # Make our blank matrix of 0's to store hot encodings.
    results = np.zeros((len(sequences), dimension))

    # For each observation and element in that observation,
    # Update the blank matrix to a 1 at row obs, column element value.
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, word_index[j]] = 1.
    return results

# I am converting the resulting giant arrays into float datatype. They are 'float64' by default, which takes up a ton of RAM!
text_onehot = vectorize_sequences(text).astype('float')

#print(train_onehot[0,0:34])
#print(reverse_index[33])
text_onehot.shape

(1600, 10275)

Don't forget we have a few other features in the data. It's not just about the review text... 

In [7]:
# One hot encoding the hotels. 
hotel_dict = {hotel : index for index, hotel in enumerate(set(trip_advisor['hotel']))}
hotels = []
for hotel in trip_advisor['hotel']:
  hotels.append(hotel_dict[hotel])

hotels_onehot = keras.utils.to_categorical(np.array(hotels))

# One hot encoding the review source
source_int = np.where(np.array(trip_advisor['source'])=='MTurk',0,np.where(np.array(trip_advisor['source'])=='TripAdvisor',1,2))
source_onehot = keras.utils.to_categorical(source_int)

# Binarizing the polarity
polarity_bin = np.where(np.array(trip_advisor['polarity'])=="negative",0,1).reshape(1600,1)

# Last step, we shuffle the data
data_onehot = np.concatenate((labels.reshape(1600,1),text_onehot,hotels_onehot,polarity_bin),axis=1)
np.random.shuffle(data_onehot)

# Then we pull out predictors and labels.
predictors = data_onehot[:,1:]
labels = data_onehot[:,0]


Now we can fit out model to the resulting data. Once again we have k-fold cross validation here. This model is incapable of learning anything much (validation accuracy never really surpasses 55-56%, and the loss is always increasing in training).

In [21]:
from tensorflow.keras import layers
import matplotlib.pyplot as plt

def build_model():
    model = keras.Sequential([
        # This is essentially a dense layer that mimics word embedding; we are reducing the one-hot encoded text (10,000+ one-hot tokens) down to 750 latent dimensions.
        layers.Dense(750, activation="linear"),
        layers.Dense(50, activation="relu",kernel_regularizer="l2"),
        layers.Dense(5, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer=keras.optimizers.Adadelta(learning_rate=0.01), loss="binary_crossentropy", metrics=[keras.metrics.BinaryAccuracy(threshold=0.5)])
    return model

model = build_model()

data_train = predictors[:1200]
labels_train = labels[:1200]
data_test = predictors[1200:]
labels_test = labels[1200:]

k = 4
num_validation_samples = len(data_train) // k
num_epochs = 50
batch_sizes = 25
all_loss_histories = []
all_val_loss_histories = []  
all_acc_histories = []
all_val_acc_histories = []
 
for fold in range(k):
    validation_data = data_train[num_validation_samples * fold:
                           num_validation_samples * (fold + 1)]
    validation_targets = labels_train[num_validation_samples * fold:
                           num_validation_samples * (fold + 1)]
    training_data = np.concatenate([
        data_train[:num_validation_samples * fold],
        data_train[num_validation_samples * (fold + 1):]])
    training_targets = np.concatenate([
        labels_train[:num_validation_samples * fold],
        labels_train[num_validation_samples * (fold + 1):]])
    model = build_model()
    history = model.fit(training_data, training_targets, 
                        validation_data = (validation_data,validation_targets), 
                        epochs=num_epochs, batch_size=batch_sizes)
    val_loss_history = history.history['val_loss']
    val_acc_history = history.history['val_binary_accuracy']
    loss_history = history.history['loss']
    acc_history = history.history['binary_accuracy']
    all_val_loss_histories.append(val_loss_history)
    all_loss_histories.append(loss_history)
    all_val_acc_histories.append(val_acc_history)
    all_acc_histories.append(acc_history)

average_loss_history = [np.mean([x[i] for x in all_loss_histories]) for i in range(num_epochs)]
average_val_loss_history = [np.mean([x[i] for x in all_val_loss_histories]) for i in range(num_epochs)]
average_acc_history = [np.mean([x[i] for x in all_acc_histories]) for i in range(num_epochs)]
average_val_acc_history = [np.mean([x[i] for x in all_val_acc_histories]) for i in range(num_epochs)]

Epoch 1/50
36/36 [==============================] - 1s 16ms/step - loss: 1.6303 - binary_accuracy: 0.5133 - val_loss: 1.6206 - val_binary_accuracy: 0.5533
Epoch 2/50
36/36 [==============================] - 0s 11ms/step - loss: 1.6058 - binary_accuracy: 0.5578 - val_loss: 1.6080 - val_binary_accuracy: 0.5533
Epoch 3/50
36/36 [==============================] - 0s 11ms/step - loss: 1.5837 - binary_accuracy: 0.5856 - val_loss: 1.5956 - val_binary_accuracy: 0.5533
Epoch 4/50
36/36 [==============================] - 0s 12ms/step - loss: 1.5611 - binary_accuracy: 0.5922 - val_loss: 1.5822 - val_binary_accuracy: 0.5700
Epoch 5/50
36/36 [==============================] - 0s 11ms/step - loss: 1.5382 - binary_accuracy: 0.6111 - val_loss: 1.5673 - val_binary_accuracy: 0.6067
Epoch 6/50
36/36 [==============================] - 0s 11ms/step - loss: 1.5154 - binary_accuracy: 0.6589 - val_loss: 1.5566 - val_binary_accuracy: 0.5800
Epoch 7/50
36/36 [==============================] - 0s 12ms/step - los

In [ ]:
# Plot validation performance. 
plt.plot(average_loss_history,c='r')
plt.plot(average_acc_history,c="r",linestyle="dashed")
plt.plot(average_val_loss_history,c='b')
plt.plot(average_val_acc_history,c='b',linestyle="dashed")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(['Training Loss','Training Accuracy','Validation Loss','Validation Accuracy'])
plt.show()